In [1]:
import pandas as pd
import tgt
import os

os.chdir("/Users/theodorelai/Documents/School/10_碩一下/00_Project/M mode")

In [2]:
def textgrid_to_csv(tgPath, # input TextGrid path
                    csvPath, # output .csv path
                    tgTier, # input TextGrid tier (please transform to zero-indexing)
                    point = None, # if it is an interval tier, please specify the point to obtain the time.
                    sort = False # If you want to sort the label by adding token numbers, set 'sort = True'.
                   ):
    try:
        tgd = tgt.read_textgrid(tgPath, encoding='utf-8', include_empty_intervals=True)
    except UnicodeDecodeError:
        tgd = tgt.read_textgrid(tgPath, encoding='utf-16', include_empty_intervals=True)
    time = []
    label = []
    annotation = []
    if type(tgd.tiers[tgTier]) == tgt.core.PointTier:
        for i in tgd.tiers[tgTier]:
            time.append(i.time)
            if sort:
                annotation.append(i.text)
                count = annotation.count(i.text)
                if count < 10:
                    label.append(i.text + '_0' + str(count))
                else:
                    label.append(i.text + '_' + str(count))
            else:
                label.append(i.text)
    elif type(tgd.tiers[tgTier]) == tgt.core.IntervalTier: 
        if point == None:
            raise ValueError("Please specify 'point'")
        if point < -0.5 or point > 1.5:
            raise ValueError("'point' should fall between -0.5 and 1.5")
        for i in tgd.tiers[tgTier]:
            if i.text not in ['', 'xxx']:
                dur = i.end_time - i.start_time
                time.append(i.start_time + dur*point)
                if sort:
                    annotation.append(i.text)
                    count = annotation.count(i.text)
                    if count < 10:
                        label.append(i.text + '_0' + str(count))
                    else:
                        label.append(i.text + '_' + str(count))
                else:
                    label.append(i.text)
    df = pd.DataFrame()
    df.insert(loc = 0, column = 'Time', value = time)
    df.insert(loc = 1, column = 'Label', value = label)
    
    df.to_csv(csvPath, index=False)
    
#     return df

In [3]:
textgrid_to_csv('Teddy_test_t_aligned.TextGrid', 'Teddy_test_t_aligned.csv', tgTier = 1, point = 1, sort = True)